In [74]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import dash_ag_grid as dag



data = pd.read_excel('C:/Users/NShuvalov/Desktop/data.xlsx')

data['Сумма'] = np.where(data['Вид движения'] == 'Приход', data['Сумма'], -data['Сумма'])

# Преобразуем строку в объект datetime с учетом времени
data['Дата'] = pd.to_datetime(data['Период'], format='%d.%m.%Y %H:%M:%S')

# Форматируем обратно в строку, если нужно
data['Дата'] = data['Дата'].dt.strftime('%d.%m.%Y %H:%M:%S')


data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y %H:%M:%S')

data['Статья учета'] = data['Статья']

expenses_df = data[data['Сумма'] < 0].copy()

expenses_df['Сумма'] = expenses_df['Сумма'].abs()

grouped_expenses = expenses_df.groupby('Статья учета')['Сумма'].sum().reset_index()
grouped_expenses_ = expenses_df.groupby(['Статья учета', 'Контрагент'])['Сумма'].sum().reset_index()
grouped_expenses_1_ = expenses_df.groupby(['Статья учета', 'Контрагент', 'Заказ'])['Сумма'].sum().reset_index()



expense_account_ids = [f"exp_{i}" for i in range(len(grouped_expenses))]

# Идентификаторы для контрагентов и заказов
expense_contractor_ids = []
expense_order_ids = []

for i in range(len(grouped_expenses)):
    # Фильтруем контрагентов для текущего счета расходов
    current_expense_type = grouped_expenses.iloc[i, 0]
    contractors = grouped_expenses_.loc[grouped_expenses_['Статья учета'] == current_expense_type]

    # Перебираем контрагентов
    for j in range(len(contractors)):
        contractor = contractors.iloc[j, 0]
        # Создаем идентификатор для каждого контрагента
        expense_contractor_ids.append(f"exp_{i}_{j}")

        # Фильтруем заказы для текущего контрагента
        orders = grouped_expenses_1_.loc[
            (grouped_expenses_1_['Статья учета'] == current_expense_type) &
            (grouped_expenses_1_['Контрагент'] == contractor)
        ]
        
        # Если заказы существуют, добавляем их идентификаторы
        if len(orders) > 0:
            for z in range(len(orders)):
                # Создаем идентификатор для каждого заказа
                expense_order_ids.append(f"exp_{i}_{j}_{z}")
        else:
            # Если заказов нет, добавляем идентификатор только для контрагента
            expense_contractor_ids.append(f"exp_{i}_{j}")

# Объединяем все идентификаторы
expenses_ids = expense_account_ids + expense_contractor_ids + expense_order_ids



expenses_parents = (
    [''] * len(grouped_expenses) +
    [f"exp_{i}" for i in range(len(grouped_expenses)) for _ in range(len(grouped_expenses_.loc[grouped_expenses_['Статья учета'] == grouped_expenses.iloc[i, 0]]))] +
    [f"exp_{i}_{j}" for i in range(len(grouped_expenses)) for j in range(len(grouped_expenses_.loc[grouped_expenses_['Статья учета'] == grouped_expenses.iloc[i, 0]]))
        for _ in range(len(grouped_expenses_1_.loc[
        (grouped_expenses_1_['Статья учета'] == grouped_expenses.iloc[i, 0]) & 
        (grouped_expenses_1_['Контрагент'] == grouped_expenses_.iloc[j, 0])
    ]))]
)


# df1 = {
#     'a': list(expenses_ids),
#     'b': list(expenses_parents)}

# df1 = pd.DataFrame(df1)
# df1
expense_order_ids

[]